In [1]:
import numpy as np

## Find voltage needed to induce 2e offset in n_g

In [12]:
## Constants
mV = 1e-3
pF = 1e-12
e = 1.60217662e-19

## Capacitance values from Chiara [https://docs.google.com/spreadsheets/d/1mLrkivUEg08DwR1by6JOdcQVONi7RsQK1ZKl2Sh9CKw/edit#gid=0]
C_TL_to_both_fins = 0.052755 * pF 
C_TL_to_fin2 = 0.036664 * pF
C_TL_to_fin3 = 0.036314 * pF


def get_cap_V(Q, C):
    return Q / C


print("On chip voltage needed to induce 2e offset charge on SQUAT: ", get_cap_V(2*e, C_TL_to_both_fins) * 1e6, "uV")
#print("On chip voltage needed to induce 2e charge on Fin 2: ", get_cap_V(2*e, C_TL_to_fin2) * 1e6, "uV")

On chip voltage needed to induce 2e offset charge on SQUAT:  6.074027561368591 uV


### Attenuation on different input lines in the BFG

In [7]:
A_atten = {
    "50K" : None,
    "4K" : 20,
    "Still" : None,
    "Cold" : 20,
    "MC" : 20
}

B_atten = {
    "50K" : None,
    "4K" : 20,
    "Still" : None,
    "Cold" : 20,
    "MC" : 10
}

D_atten = {
    "50K" : None,
    "4K" : 20,
    "Still" : None,
    "Cold" : 1,
    "MC" : None
}


## Look at what warm voltages are necessary for each line type.  Calc heat dissipationn on MC

In [49]:
def dB_to_ratio(dB):
    return 10**(dB/20)

def calc_voltage_on_chip(V_in, atten):
    cumulative_atten = 1
    for stage, dB in atten.items():
        if dB is not None:
            cumulative_atten *= dB_to_ratio(dB)
        if stage == "MC" and dB is not None:
            print("Attenuation at MC stage: ", dB)
            V_before_mc = V_in / cumulative_atten
            V_after_mc = V_before_mc * dB_to_ratio(dB)
            heat_dissipated = (V_before_mc - V_after_mc)**2 / 50
            print("Heat dissipated on MC: ", heat_dissipated*1e6, "uW")
        elif stage == "MC":
            print("No attenuation at MC stage, can't calculate heat dissipated")
    V_out = V_in / cumulative_atten
    return V_out

## Run for each line
V_in = 10 * mV
V_out = calc_voltage_on_chip(V_in, A_atten)
print("Voltage on chip for input line A: ", V_out*1e6, "uV \n\n")    

V_in = 3.2 * mV
V_out = calc_voltage_on_chip(V_in, B_atten)
print("Voltage on chip for input line B: ", V_out*1e6, "uV \n\n")    

V_in = 0.1 * mV
V_out = calc_voltage_on_chip(V_in, D_atten)
print("Voltage on chip for input line D: ", V_out*1e6, "uV \n\n")    

Attenuation at MC stage:  20
Heat dissipated on MC:  0.00016200000000000003 uW
Voltage on chip for input line A:  10.0 uV 


Attenuation at MC stage:  10
Heat dissipated on MC:  9.575310703950318e-06 uW
Voltage on chip for input line B:  10.119288512538814 uV 


No attenuation at MC stage, can't calculate heat dissipated
Voltage on chip for input line D:  8.912509381337456 uV 


